In [144]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict
import string

import re

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
popular_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/popular.json")
assert len(popular_examples) == 1
popular_examples = popular_examples[0]

random_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/random.json")
assert len(random_examples) == 1
random_examples = random_examples[0]

recent_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/recent.json")
assert len(recent_examples) == 1
recent_examples = recent_examples[0]

ripple_edits_examples = recent_examples #  +  # random_examples #  + random_examples
len(ripple_edits_examples)

1948

In [44]:
0.15 * len(ripple_edits_examples)

292.2

In [45]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
for example in ripple_edits_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    # non_zero_outerloop_count += len(outerloop_instances) > 0
    if len(outerloop_instances) > 0 and len(locality_instances) > 0:
        
        efficacy_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        efficacy_queries = [q for q in efficacy_queries if len(q["answers"]) > 0]
        efficacy_queries = [q for q in efficacy_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in efficacy_queries])
        
        if len(efficacy_queries) == 0:
            continue
        
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        strong_meta_examples.append(example)
    elif len(locality_instances) == 0:
        
        efficacy_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        efficacy_queries = [q for q in efficacy_queries if len(q["answers"]) > 0]
        efficacy_queries = [q for q in efficacy_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in efficacy_queries])
        
        if len(efficacy_queries) == 0:
            continue
        
        weak_meta_examples.append(example)
    else:
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        weak_meta_examples.append(example)
        
print("# strong examples:", len(strong_meta_examples))
print("# weak examples:", len(weak_meta_examples))
print("# invalid:", len(ripple_edits_examples) - len(strong_meta_examples) - len(weak_meta_examples))

# strong examples: 1318
# weak examples: 575
# invalid: 55


In [47]:
200 / len(ripple_edits_examples)

0.1026694045174538

In [48]:
n_test = 200
n_valid = 200
np.random.shuffle(strong_meta_examples)
np.random.shuffle(weak_meta_examples)

test_examples = strong_meta_examples[:n_test]
valid_examples = strong_meta_examples[n_test:n_test+n_valid]
train_examples = strong_meta_examples[n_test+n_valid:] + weak_meta_examples
np.random.shuffle(train_examples)

In [51]:
# subset = "recent"

# os.makedirs(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}", exist_ok=True)

In [52]:
# io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/train.jsonl")
# io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/valid.jsonl")
# io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/test.jsonl")

### merge the train/dev/test for all subsets

In [200]:
train_examples = []
valid_examples = []
test_examples = []
for subset in ["popular", "recent", "random"]:
    train_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/train.jsonl")
    valid_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/valid.jsonl")
    test_examples += io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/{subset}/test.jsonl")
print("# train examples:", len(train_examples))
print("# valid examples:", len(valid_examples))
print("# test examples:", len(test_examples))

# train examples: 3686
# valid examples: 500
# test examples: 500


In [ ]:
# os.makedirs(f"{vars.DATA_DIR}/ripple_edits/meta_train/all", exist_ok=True)
# io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/train.jsonl")
# io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/valid.jsonl")
# io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/test.jsonl")

In [ ]:
# recent_popular_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train.jsonl")

In [ ]:
# random_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_random/train.jsonl")

In [ ]:
# io.dump_jsonlines(recent_popular_train + random_train, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train_w_random.jsonl")

### Check spurious correlation in dataaset (answer verbtaim in edit)

In [139]:
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/recent/train.jsonl")

In [140]:
len(train_examples)

1493

In [141]:
non_zero_outerloop_count = 0
meta_examples = []
propagation2substring_count = defaultdict(int)
propagation2verbtaim_count = defaultdict(int)
propagation2count = defaultdict(int)
# strong_meta_examples = []
# weak_meta_examples = []
count_rel = defaultdict(int)
count_prop_types = defaultdict(int)

for example in train_examples[:]:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    example["edit"]["text"] = example["edit"]["prompt"]
    meta_examples.append(example["edit"])
    
    outerloop_queries = []
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    propagation2substring_count[k] += int(any(a in q["prompt"] for a in ans_candidates))
                    outerloop_queries.append(q)
    for outer_q in outerloop_queries:
        count_prop_types["efficacy::"+ outer_q["question_type"]] += 1
        count_rel["efficacy::"+ outer_q["relation"]] += 1
    # assert len(outerloop_queries) > 0
    meta_examples.extend(outerloop_queries)
    
    locality_queries = []
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    propagation2substring_count[k] += int(any(a in q["prompt"] for a in ans_candidates))
                    locality_queries.append(q)
    for loc_q in locality_queries:
        count_prop_types["specificity::"+ loc_q["question_type"]] += 1
        count_rel["specificity::"+ loc_q["relation"]] += 1
    
    meta_examples.extend(locality_queries)
    
    # assert len(locality_queries) > 0
        
# print("# strong examples", len(strong_meta_examples))
# print("# weak examples", len(weak_meta_examples))

In [143]:
propagation2count

defaultdict(int,
            {'Logical_Generalization': 813,
             'Compositionality_I': 7037,
             'Subject_Aliasing': 892,
             'Relation_Specificity': 5509,
             'Forgetfulness': 951,
             'Compositionality_II': 751})

In [129]:
{k: round(propagation2verbtaim_count[k] / propagation2count[k] * 100, 1) for k in propagation2count}

{'Subject_Aliasing': 100.0,
 'Relation_Specificity': 1.1,
 'Logical_Generalization': 58.5,
 'Compositionality_I': 2.9,
 'Forgetfulness': 0.0,
 'Compositionality_II': 100.0}

In [32]:
119- 81

38

In [30]:
propagation2substring_count

defaultdict(int,
            {'Logical_Generalization': 9,
             'Relation_Specificity': 8,
             'Compositionality_I': 28,
             'Compositionality_II': 10,
             'Subject_Aliasing': 1,
             'Forgetfulness': 81})

In [29]:
{k: round(propagation2substring_count[k] / propagation2count[k] * 100, 1) for k in propagation2count}

{'Logical_Generalization': 7.4,
 'Relation_Specificity': 0.9,
 'Compositionality_I': 2.8,
 'Compositionality_II': 9.4,
 'Subject_Aliasing': 0.3,
 'Forgetfulness': 68.1}

In [4]:
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}.jsonl")

In [5]:
len(ds)
len(test_data)

3686

In [109]:
ds

Dataset({
    features: ['prompt', 'subject_id', 'relation', 'target_id', 'original_fact', 'context', 'paraphrase', 'object', 'subject'],
    num_rows: 500
})

In [311]:
ent_label2id = io.load_json("/data/users/zliu/RippleEdits/src/wikidata/ent_label2id.json")

In [543]:
split = "test"
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}.jsonl")

In [544]:
# ent_label2id["Enrico Chiesa"] = "Q345026"
# ent_label2id["Jimmy Carter"] = "Q23685"
# ent_label2id["Adam Rich"] = "Q1762781"
# ent_label2id["Croatia national association football team"] = "Q134479"
# ent_label2id[""] = "Q16083337"
# ent_label2id["Milivoje 'Misko' Milojevic"] = "Q108063841"
# ent_label2id["field hockey at the 2020 Summer Olympics"] = "Q32646403"

id2ent_label = {v: k for k, v in ent_label2id.items()}
# ent_label2id["Amānullāh Khān"] == "Q153620"
# id2ent_label["Q153620"] = "Amānullāh Khān"
# ent_label2id["Princess Amöena Amalie of Bentheim-Tecklenburg"] = "Q22077731"
# id2ent_label["Q22077731"] = "Princess Amöena Amalie of Bentheim-Tecklenburg"
# assert len(ent_label2id) == len(id2ent_label)

In [527]:
# ent_label2id["Princess Amöena Amalie of Bentheim-Tecklenburg"]

In [528]:
# id2ent_label["Q134479"]

In [529]:
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.entity import WikidataItem
import functools

@functools.lru_cache()
def wikidata_item_given_id(ent_id: str):
    try:
        return WikidataItem(get_entity_dict_from_api(ent_id))
    except:
        return None

def get_label(ent_id: str):
    if isinstance(ent_id, list):
        if len(ent_id) > 0:
            ent_id = ent_id[0]
        else:
            return ent_id
    if ent_id[0] != 'Q':
        return ent_id
    item = wikidata_item_given_id(ent_id)
    if item is not None:
        label = item.get_label()
    else:
        return ent_id
    if label is None:
        return ent_id
    return label

In [547]:
prompt2subjtarg = io.load_json("/data/users/zliu/mend/notebooks/prompt2subject_target.json")

# for i in manual_process_ids:
#     if test_data[i]["edit"]["prompt"] in prompt2subjtarg:
#         continue
#     print(i)
#     print(test_data[i]["edit"]["prompt"])
#     subject_id = test_data[i]["edit"]["subject_id"]
#     target_id = test_data[i]["edit"]["target_id"]
#     print(subject_id, "==", id2ent_label[test_data[i]["edit"]["subject_id"]], "@@@", target_id, "==", id2ent_label[test_data[i]["edit"]["target_id"]])
#     print()

In [ ]:
ds = load_from_disk("/data/users/zliu/KE-by-CP/data/ripple_edits/meta_train/all/train_w_subjtarget.hf")
train_prompt2subjtarg = {}
for i in range(len(ds)):
    original_test = deepcopy(ds[i])
    assert "subject_id" in original_test
    assert "target_id" in original_test
    subject = original_test["subject"]
    target = original_test["object"]
        
    train_prompt2subjtarg[original_test["prompt"]] = [subject, target]

In [531]:
# prompt2subjtarg["The place of birth of Eddie Van Halen is Caucasia."] = ["Eddie Van Halen", "Caucasia"]
# prompt2subjtarg["The name of the country which Croatia national association football team is associated with is Moscow."] = ["Croatia national association football team", "Moscow"]



In [555]:
# io.dump_json(prompt2subjtarg, "/data/users/zliu/mend/notebooks/prompt2subject_target.json")

In [556]:
# prompt2subjtarg = train_prompt2subjtarg

In [577]:
new_test_data = []
manual_process_ids = []
count = 0
for i in tqdm(range(len(test_data))):
    original_test = deepcopy(test_data[i])
    assert "subject_id" in original_test["edit"]
    assert "target_id" in original_test["edit"]
    
    subject_id = original_test["edit"]["subject_id"]
    target_id = original_test["edit"]["target_id"]
    if original_test["edit"]["prompt"] in prompt2subjtarg:
        subject, target = prompt2subjtarg[original_test["edit"]["prompt"]]
        original_test["edit"]["subject"] = subject
        original_test["edit"]["target"] = target
        new_test_data.append(original_test)
        continue
    elif original_test["edit"]["prompt"] == "The name of the architect of  is Aleksei Benois.":
        original_test["edit"]["prompt"] = "The name of the architect of Palace of the Emir of Bukhara is Aleksei Benois."
        original_test["edit"]["subject"] = "Palace of the Emir of Bukhara"
        original_test["edit"]["target"] = "Aleksei Benois"
        new_test_data.append(original_test)
        continue
    elif original_test["edit"]["prompt"] == "The name of the field of work of Michael H. Hart is astronomy.":
        original_test["edit"]["subject"] = "Michael H. Hart"
        original_test["edit"]["target"] = "astronomy"
        new_test_data.append(original_test)
        continue
    elif original_test["edit"]["prompt"] == "The name of the screenwriter of  is Laurent-Frédéric Bollée.":
        original_test["edit"]["prompt"] = "The name of the screenwriter of The bomb : the weapon that changed the world is Laurent-Frédéric Bollée."
        original_test["edit"]["subject"] = "The bomb : the weapon that changed the world"
        original_test["edit"]["target"] = "Laurent-Frédéric Bollée"
        new_test_data.append(original_test)
        continue
    elif original_test["edit"]["prompt"] == "The name of the director of  is Milivoje 'Misko' Milojevic.":
        original_test["edit"]["subject"] = ""
        original_test["edit"]["target"] = "Milivoje 'Misko' Milojevic"
        new_test_data.append(original_test)
        continue
    
    if subject_id in id2ent_label and target_id in id2ent_label:
        subject = id2ent_label[subject_id]
        target = id2ent_label[target_id]
    else:
        subject = get_label(subject_id)
        target = get_label(target_id)
       
        
    
    if subject not in original_test["edit"]["prompt"] or target not in original_test["edit"]["prompt"] or target + "." not in original_test["edit"]["prompt"]:
        manual_process_ids.append(i)
        continue
    
    original_test["edit"]["subject"] = subject
    original_test["edit"]["target"] = target
    assert target in original_test["edit"]["prompt"]
    assert target + "." in original_test["edit"]["prompt"], original_test["edit"]["prompt"] + "@@@" + target
    # create the context of the prompt
    original_test["edit"]["context"] = original_test["edit"]["prompt"][::-1].replace((target + ".")[::-1], "", 1)[::-1].strip()
    
    new_test_data.append(original_test)
    ent_label2id[subject] = subject_id
    ent_label2id[target] = target_id
    id2ent_label[subject_id] = subject
    id2ent_label[target_id] = target
    # assert subject_id in id2ent_label, original_test["edit"]
    # assert target_id in id2ent_label

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:00<00:00, 902.95it/s]


In [558]:
len(new_test_data) == len(test_data)

True

In [550]:
manual_process_ids

[198]

In [521]:
# len(manual_process_ids)

In [522]:
# io.dump_jsonlines(manual_process_ids, "/data/users/zliu/mend/notebooks/manual_process_ids.jsonl")

In [542]:
## Sanity check the data
faulty_ids = []
aug_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/valid_aug.jsonl")
for i, d in enumerate(aug_data):
    assert "subject" in d["edit"]
    assert "target" in d["edit"]
    if d["edit"]["subject"] not in d["edit"]["prompt"] or d["edit"]["target"] not in d["edit"]["prompt"] or d["edit"]["target"] + "." not in d["edit"]["prompt"]:
        faulty_ids.append(i)
len(faulty_ids)

0

In [552]:
# prompt2subjtarg["The name of the award Designated Survivor won is Honour Roll Clasp of the Army."] = ["Designated Survivor", "Honour Roll Clasp of the Army"]
# prompt2subjtarg["The occupation of Ali Daei is alcalde / alcaldesa."] = ["Ali Daei", "alcalde / alcaldesa"]
# prompt2subjtarg["The name of the country which Jana Gana Mana is associated with is Yamataikoku."] = ["Jana Gana Mana", "Yamataikoku"]
# prompt2subjtarg["The place of birth of Lisa Welp is United States of America."] = ["Lisa Welp", "United States of America"]
# prompt2subjtarg["The occupation of Karl Urban is aircrew member."] = ["Karl Urban", "aircrew member"]
# prompt2subjtarg["The name of the alma mater of  is United States of America."] = ["", "United States of America"]
# prompt2subjtarg["The place of burial of Jim Fixx is United States of America."] = ["Jim Fixx", "United States of America"]
# prompt2subjtarg["The place of birth of Krystyna Swinarska is United States of America."] = ["Krystyna Swinarska", "United States of America"]
# prompt2subjtarg["The place of death of Gwynne Evans is United States of America."] = ["Gwynne Evans", "United States of America"]
# prompt2subjtarg["The place of birth of Mohamed Zakariya is United States of America."] = ["Mohamed Zakariya", "United States of America"]
# prompt2subjtarg["The name of the ethnic group which Mike Hill is associated with is British people."] = ["Mike Hill", "British people"]
# prompt2subjtarg["The place of birth of Janet A. Anderson is United States of America."] = ["Janet A. Anderson", "United States of America"]
# prompt2subjtarg["The name of the country which American-Cassinese Benedictine Congregation is associated with is Yamataikoku."] = ["American-Cassinese Benedictine Congregation", "Yamataikoku"]
# prompt2subjtarg["The name of the country which Seed Farm is associated with is Iowa River."] = ["Seed Farm", "Iowa River"]
# prompt2subjtarg["The name of the country which Kimré is associated with is Yamataikoku."] = ["Kimré", "Yamataikoku"]
# prompt2subjtarg["The occupation of Wanda O Jacobs is acting rank."] = ["Wanda O Jacobs", "acting rank"]
# prompt2subjtarg["The occupation of David Vejražka is newspaper delivery person."] = ["David Vejražka", "newspaper delivery person"]
# prompt2subjtarg["The name of the position held by Lucien M. Gex is Mayor of Dayton, Ohio."] = ["Lucien M. Gex", "Mayor of Dayton, Ohio"]
# prompt2subjtarg["The name of the country which Ankalagi is associated with is Yamataikoku."] = ["Ankalagi", "Yamataikoku"]
# prompt2subjtarg["The name of the continent which Sierra Carbonera is part of is Australian continent."] = ["Sierra Carbonera", "Australian continent"]
# prompt2subjtarg["The place of birth of F. Hudson Miller is United States of America."] = ["F. Hudson Miller", "United States of America"]
# prompt2subjtarg["The place of birth of Margaret Bamgbose is United States of America."] = ["Margaret Bamgbose", "United States of America"]
# prompt2subjtarg["The name of the ethnic group which Richard Smalbroke is associated with is British people."] = ["Richard Smalbroke", "British people"]
prompt2subjtarg["The name of the ethnic group which Michael J. Knowles is associated with is Italian Americans."] = ["Michael J. Knowles", "Italian Americans"]

In [553]:
for i in manual_process_ids:
    if test_data[i]["edit"]["prompt"] in prompt2subjtarg:
        continue
    print(i)
    print(test_data[i]["edit"]["prompt"])
    subject_id = test_data[i]["edit"]["subject_id"]
    target_id = test_data[i]["edit"]["target_id"]
    print(subject_id, "==", id2ent_label[test_data[i]["edit"]["subject_id"]], "@@@", target_id, "==", id2ent_label[test_data[i]["edit"]["target_id"]])
    print()

In [458]:
# test_data[884]

In [277]:
# len(new_test_data) == len(test_data)
# aug_data[10]

In [ ]:
# split = "train"

In [218]:
# len(io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}.jsonl")) == len(io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl"))

In [339]:
for i in manual_process_ids:
    
    if test_data[i]["edit"]["prompt"] in prompt2subjtarg:
        continue
    print(i)
    print(test_data[i]["edit"]["prompt"])
    subject_id = test_data[i]["edit"]["subject_id"]
    target_id = test_data[i]["edit"]["target_id"]
    print(subject_id, "==", id2ent_label[test_data[i]["edit"]["subject_id"]], "@@@", target_id, "==", id2ent_label[test_data[i]["edit"]["target_id"]])
    print()

In [562]:
# split

In [561]:
# io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")

In [ ]:
# io.dump_json(ent_label2id, "/data/users/zliu/RippleEdits/src/wikidata/ent_label2id.json")

In [76]:
# original_test["edit"]

### find instance with empty subject

In [3]:
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/test_aug.jsonl")
faulty_ids = []
for i, d in enumerate(test_data):
    if d["edit"]["subject"] == "" or d["edit"]["target"] == "":
        faulty_ids.append(i)
len(faulty_ids)

3

In [6]:
for i in faulty_ids:
    
    # if test_data[i]["edit"]["prompt"] in prompt2subjtarg:
        # continue
    print(i)
    print(test_data[i]["edit"]["prompt"])
    subject_id = test_data[i]["edit"]["subject_id"]
    target_id = test_data[i]["edit"]["target_id"]
    # print(subject_id, "==", id2ent_label[test_data[i]["edit"]["subject_id"]], "@@@", target_id, "==", id2ent_label[test_data[i]["edit"]["target_id"]])
    print()

145
The name of the director of  is Margarete Wallmann.

279
The name of the composer of  is Ludwig van Beethoven.

281
The name of the founder of  is Henry Bauër.



In [8]:
test_data[145]

{'example_type': 'recent',
 'edit': {'prompt': 'The name of the director of  is Margarete Wallmann.',
  'subject_id': 'Q47477080',
  'relation': 'DIRECTOR',
  'target_id': 'Q214024',
  'subject': '',
  'target': 'Margarete Wallmann',
  'context': 'The name of the director of  is',
  'paraphrase': "The director's name is"},
 'Logical_Generalization': [],
 'Compositionality_I': [{'test_queries': [{'prompt': 'The gender of the director of hola is',
     'answers': [{'value': 'female',
       'aliases': ['woman',
        'human female',
        'female person',
        'lady',
        'female human',
        'fairer sex',
        'female gender',
        'fem',
        '♀',
        'f',
        'women',
        'girl',
        'girls',
        'female character']}],
     'query_type': 'two_hop',
     'subject_id': 'Q47477080',
     'relation': 'DIRECTOR',
     'target_ids': ['Q214024'],
     'phrase': 'The gender of the director of hola is',
     'second_relation': 'SEX_OR_GENDER',
     's

In [80]:
# id2ent_label["Q16083337"], id2ent_label["Q108063841"]

In [78]:
# test_data[1908]["edit"]

In [79]:
# len(manual_process_ids)

### getting augmented dataset with llm

In [ ]:
split = "train"
ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_w_prefix.hf")
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}.jsonl")

new_test_data = []

count = 0
for i in range(len(test_data)):
    original_test = test_data[i]
    augmentation = ds[i-count]
    if original_test["edit"]["prompt"] == "The name of the award Ravi Kumar Dahiya won is Obaland  Awards.":
        # print("found")
        original_test["edit"]["subject"] = "The name of the award"
        original_test["edit"]["object"] = "Obaland  Awards"
        assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
        assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
        new_test_data.append(original_test)
        count += 1
        continue
        # print(original_test)
        # print(augmentation)
    elif original_test["edit"]["prompt"] == "The occupation of Sandeep  Singh is Steward's assistant.":
        # print("found")
        original_test["edit"]["subject"] = "Sandeep  Singh"
        original_test["edit"]["object"] = "Steward's assistant"
        assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
        assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
        new_test_data.append(original_test)
        count += 1
        continue
    elif original_test["edit"]["prompt"] == "The gender of Wu  Xinghuan is takatāpui.":
        # print("found")
        original_test["edit"]["subject"] = "Wu  Xinghuan"
        original_test["edit"]["object"] = "takatāpui"
        assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
        assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
        new_test_data.append(original_test)
        count += 1
        continue
    else:
        assert original_test["edit"]["prompt"] == augmentation["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["prompt"]
        assert augmentation["subject"] in original_test["edit"]["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["subject"]
        assert augmentation["object"] in original_test["edit"]["prompt"]

        original_test["edit"]["subject"] = augmentation["subject"]
        original_test["edit"]["object"] = augmentation["object"]
        new_test_data.append(original_test)
    

In [32]:
count

3

In [33]:
test_data[393]["edit"]["prompt"]

'The name of the league which Paul Westhead plays in is Southern Counties East Football League.'

In [595]:
# test_data[-3]

In [ ]:
# io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")


### Prepare a ripple edits for MEND-original version (before generating paraphrase)

In [584]:
split = "train"
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")
# train_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_w_prefix.hf")
# train_examples[0]

In [585]:
train_examples_w_context = []
for i, example in enumerate(train_examples):
    new_example = deepcopy(example)
    obj = new_example["edit"]["target"]
    prompt = new_example["edit"]["prompt"]
    
    assert obj in new_example["edit"]["prompt"]
    assert obj + "." in new_example["edit"]["prompt"], new_example["edit"]["prompt"] + "@@@" + obj
    
    new_example["edit"]["context"] = prompt[::-1].replace((obj + ".")[::-1], "", 1)[::-1].strip()
    train_examples_w_context.append(new_example)


### Prepare a ripple edits for MEND-original version (after generating paraphrase)

In [632]:
split = "test"
ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_w_paraphrase.hf")
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")
new_test_data = []
for i in range(len(ds)):
    augmentation = deepcopy(ds[i])
    new_test_datum = deepcopy(train_examples[i])
    assert new_test_datum["edit"]["context"] == augmentation["context"], str(i) + "@@@" + str(augmentation["context"])
    new_test_datum["edit"]["paraphrase"] = augmentation["paraphrase"]
    new_test_data.append(new_test_datum)

FileNotFoundError: [Errno 2] No such file or directory: '/u/zliu/datastor1/KE-by-CP/data/ripple_edits/meta_train_recent+popular/test_aug.jsonl'

In [633]:
len(new_test_data) == len(ds)

True

In [ ]:
# new_test_data[0]["edit"]

{'prompt': 'The name of the continent which United Arab Emirates is part of is Indian Ocean.',
 'subject_id': 'Q878',
 'relation': 'CONTINENT',
 'target_id': 'Q1239',
 'original_fact': {'prompt': 'The name of the continent which United Arab Emirates is part of is Asia.',
  'subject_id': 'Q878',
  'relation': 'CONTINENT',
  'target_id': 'Q48'},
 'subject': 'United Arab Emirates',
 'target': 'Indian Ocean',
 'context': 'The name of the continent which United Arab Emirates is part of is',
 'paraphrase': 'The continent that includes the United Arab Emirates is named'}

In [635]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")

In [619]:
# train_examples[61]["edit"]["prompt"]

In [588]:
# train_examples_w_context[0]["edit"]

In [587]:
# io.dump_jsonlines(train_examples_w_context, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")

# special treatment for test set

In [ ]:

new_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    new_example = deepcopy(example)
    
    new_example["edit"]["context"] = augmentation["context"]
    new_example["edit"]["paraphrase"] = augmentation["paraphrase"]
    new_example["edit"]["object"] = augmentation["object"]
    new_examples.append(new_example)
    # if example["edit"]["prompt"] != augmentation["prompt"]:
    #     mismatch_c += 1
    # else:
        
    #     assert example["edit"]["prompt"] == augmentation["prompt"]
    #     assert augmentation["object"] in example["edit"]["prompt"]
    
    #     flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["object"], "paraphrase": augmentation["paraphrase"]})
    

In [665]:
# if split == "test":
#     io.dump_jsonlines(new_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_mend.jsonl")
split = "valid"
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_aug.jsonl")

In [666]:
# train_examples[0]

In [667]:
flattend_train_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    # augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    
    # if example["edit"]["prompt"] != augmentation["prompt"]:
        # mismatch_c += 1
    # else:
        
    # assert example["edit"]["prompt"] == augmentation["prompt"]
    assert example["edit"]["target"] in example["edit"]["prompt"], example["edit"]["target"] + "@@@" + example["edit"]["prompt"]

    flattend_train_examples.append({"context": example["edit"]["context"], "completion": example["edit"]["target"], "paraphrase": example["edit"]["paraphrase"]})
    
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "prompt": q["prompt"].strip() + " " + ans_candidates[0]})
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "prompt": q["prompt"].strip() + " " + ans_candidates[0]})
    

In [671]:
len(missing_examples)

2254

In [669]:
len(flattend_train_examples)

3643

In [670]:
flattend_train_examples[0]

{'context': 'The name of the award Henry Cavill won is',
 'completion': 'Bologna Ragazzi Award',
 'paraphrase': 'The title of the accolade that Henry Cavill received is'}

In [ ]:
# io.dump_jsonlines(missing_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_missing_parahrase.jsonl")


In [673]:
missing_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_missing_parahrase_w_paraphrase.hf")
missing_train_examples = []
for i in range(len(missing_ds)):
    augmentation = missing_ds[i]
    
    missing_train_examples.append({"context": augmentation["context"], "completion": augmentation["completion"], "paraphrase": augmentation["paraphrase"]})

In [674]:
len(flattend_train_examples + missing_train_examples)

5897

In [ ]:
# len(flattend_train_examples)


17096

In [ ]:
# io.dump_jsonlines(flattend_train_examples + missing_train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/all/{split}_mend.jsonl")

In [ ]:
# for ex in io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/valid_mend.jsonl"):
#     assert "context" in ex and ex["context"] is not None
#     assert "completion" in ex and ex["completion"] is not None
#     assert "paraphrase" in ex and ex["paraphrase"] is not None

In [11]:
converted_recoe_aggregation[-1]
len(converted_recoe_aggregation)

508

### Integrate RoCE data to ripple edit trainings

In [116]:
recoe_aggregation = io.load_json("/u/zliu/datastor1/ReCoE/data/aggregation/counterfactual_datapoints_verified_atomic.json")

converted_recoe_aggregation = [
    {
        "texts": " ".join([f["fact"] for f in x["direct_counterfactual_fact"]]),
        "question": x["question"],
        "answer": x["counterfactual_answer"]
     }
    for x in recoe_aggregation
]
assert len(converted_recoe_aggregation) == len(recoe_aggregation)


In [117]:
recoe_comparative = io.load_json("/u/zliu/datastor1/ReCoE/data/comparative/counterfactual_datapoints_verified_atomic.json")
converted_recoe_comparative = []
for x in recoe_comparative:
    assert "choice_1_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_2_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_3_counterfactuals" not in x["counterfactuals_per_choice"]
    
    counter_fact1_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_1_counterfactuals"]]
    counter_fact2_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_2_counterfactuals"]]
    texts = [counter_fact1_text, counter_fact2_text]
    np.random.shuffle(texts)
    counter_fact1_text_after_rand, counter_fact2_text_after_rand = texts
    
    
    converted_recoe_comparative.append(
        {
            "texts": " ".join(counter_fact1_text_after_rand + counter_fact2_text_after_rand),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(converted_recoe_comparative) == len(recoe_comparative)

In [118]:
# [len(x["facts_per_choice"]) for x in recoe_comparative]
recoe_counting = io.load_json("/u/zliu/datastor1/ReCoE/data/counting/counterfactual_datapoints_verified_atomic.json")
converted_recoe_counting = []
for x in recoe_counting:
    converted_recoe_counting.append(
        {
            "texts": " ".join([f["sentence"] for f in x["counterfactual_facts"]]),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(recoe_counting) == len(converted_recoe_counting)

In [119]:
recoe_sorting = io.load_json("/u/zliu/datastor1/ReCoE/data/sorting/counterfactual_datapoints_verified_atomic.json")
converted_recoe_sorting = []
for x_i, x in enumerate(recoe_sorting):
    assert "counterfactuals_per_choice" in x
    n_facts = len([k for k in x["facts_per_choice"].keys() if "choice_" in k])
    texts = []
    count = 0
    # assert n_facts == 3
    counter_factuals = x["counterfactuals_per_choice"]
    factuals = x["facts_per_choice"]
    
    unchanged_facts_added = False
    for f_i in range(1, n_facts + 1):
        
        if f"choice_{f_i}_counterfactuals" not in counter_factuals:
            if not unchanged_facts_added:
                texts.extend(counter_factuals["unchanged_facts"])
            else:
                continue
        else:
            texts.extend([f["fact"] for f in counter_factuals[f"choice_{f_i}_counterfactuals"]])
    converted_recoe_sorting.append(
        {
            "texts": " ".join(texts),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(converted_recoe_sorting) == len(recoe_sorting)

In [120]:
recoe_subtraction = io.load_json("/u/zliu/datastor1/ReCoE/data/subtraction/counterfactual_datapoints_verified_atomic.json")


converted_recoe_subtraction = [
    {
        "texts": " ".join([f["fact"] for f in x["direct_counterfactual_fact"]]),
        "question": x["question"],
        "answer": x["counterfactual_answer"]
     }
    for x in recoe_subtraction
]
assert len(recoe_subtraction) == len(converted_recoe_subtraction)


In [ ]:
recoe_superlative = io.load_json("/u/zliu/datastor1/ReCoE/data/superlative/counterfactual_datapoints_verified_atomic.json")

converted_recoe_superlative = []
for x_i, x in enumerate(recoe_superlative):
    assert "counterfactuals_per_choice" in x
    n_facts = len([k for k in x["facts_per_choice"].keys() if "choice_" in k])
    texts = []
    count = 0
    # assert n_facts == 3
    counter_factuals = x["counterfactuals_per_choice"]
    factuals = x["facts_per_choice"]
    
    unchanged_facts_added = False
    for f_i in range(1, n_facts + 1):
        
        if f"choice_{f_i}_counterfactuals" not in counter_factuals:
            if not unchanged_facts_added:
                texts.extend(counter_factuals["unchanged_facts"])
            else:
                continue
        else:
            texts.extend([f["fact"] for f in counter_factuals[f"choice_{f_i}_counterfactuals"]])
    converted_recoe_superlative.append(
        {
            "texts": " ".join(texts),
            "question": x["question"],
            "answer": x["counterfactual_answer"]
        }
    )
assert len(recoe_superlative) == len(converted_recoe_superlative)
    # assert count == 2, x

In [122]:
converted_recoe = converted_recoe_aggregation + converted_recoe_comparative + converted_recoe_counting + converted_recoe_sorting + converted_recoe_subtraction + converted_recoe_superlative

In [130]:
converted_recoe = [
    {
        "texts": x["texts"],
        "question": x["question"],
        "answer": str(x["answer"])
    }
    for x in converted_recoe
]

In [131]:
all([isinstance(x["answer"], str) for x in converted_recoe])

True

In [132]:
io.dump_jsonlines(converted_recoe, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/recoe.jsonl")

In [133]:
recent_popular_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train.jsonl")

In [134]:
io.dump_jsonlines(recent_popular_train + converted_recoe, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train_w_recoe.jsonl")